<a href="https://colab.research.google.com/github/abbeymars/AbigailMarsella-AER850Project2/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import zipfile
import os

#Step 1: Data Processing
#define input image shape
#input_shape = (500, 500, 3)  # Image dimensions: 500x500 pixels, 3 color channels (RGB)
#batch_size = 32

#establish the train and validation data directory
#mount google drive
from google.colab import drive
drive.mount('/content/drive')
zip_file = '/content/drive/My Drive/AER850Proj2/Dataset.zip'
extract = '/content/Dataset'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract)
drive.mount("/content/drive", force_remount=True)

#establish train, validation, and test directories
train = os.path.join(extract, 'Train')
validation = os.path.join(extract, 'Validation')
test = os.path.join(extract, 'Test')









Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive
